# Introduction to Linear Programming with Python - Part 5
## Using PuLP with pandas and binary constraints to solve a scheduling problem

In this example, we'll be solving a scheduling problem. We have 5 pilots and 10 different go's to fill

We want to produce a schedule of pilots from both plants that meets our demand with the lowest cost.

A pilot can be in 2 states:
* Off - not flying
* On - flying

Pilots are either available or not available for each go.

Goal is to fill the schedule with each pilot getting as few flights as possible

In [1]:
import pandas as pd
import pulp
#import math

In [2]:
pilots = pd.read_csv('csv/pilot_availability_v2.csv',index_col=['PILOT'])
pilots

,1,2,3,4,5,6,7,8,9
PILOT,,,,,,,,,
A,0,1,0,1,1,0,1,0,0
B,0,1,0,1,1,0,1,1,1
C,1,0,1,1,1,0,1,1,1
D,0,0,1,1,0,1,0,0,0
E,1,0,1,0,1,0,1,0,0


In [3]:
pilot_quals = pd.read_csv('csv/pilot_qual.csv',index_col=['PILOT'])
pilot_quals

,QUAL
PILOT,
A,2
B,1
C,2
D,1
E,2


We'll also import our demand data

In [19]:
lines = pd.read_csv('csv/go_demand.csv', index_col=['GO','Line'])
lines

Requirement
GO Line             
1  101             2
   102             1
2  201             2
3  301             2
   302             1
4  401             2
5  501             2
   502             1
6  601             2
7  701             2
   702             1
8  801             2
9  901             2
   902             1

In [21]:
lines.columns

Index(['Requirement'], dtype='object')

Pilot status is modelled as a binary variable. It will have a value of 1 if the pilot is flying and a value of 0 when the pilot is off.

Binary variables are the same as integer variables but constrained to be >= 0 and <=1

Again this has a value for each month for each factory, again given by the index of our DataFrame

In [5]:
type(pilots)

pandas.core.frame.DataFrame

In [6]:
pilot_status = pulp.LpVariable.dicts("pilot_status",
                                     ((Line,PILOT) for Line  in lines.index for PILOT in pilots.index ),
                                     cat='Binary')

In [7]:
pilot_status

{((1, 101), 'A'): pilot_status_((1,_101),_'A'),
 ((1, 101), 'B'): pilot_status_((1,_101),_'B'),
 ((1, 101), 'C'): pilot_status_((1,_101),_'C'),
 ((1, 101), 'D'): pilot_status_((1,_101),_'D'),
 ((1, 101), 'E'): pilot_status_((1,_101),_'E'),
 ((1, 102), 'A'): pilot_status_((1,_102),_'A'),
 ((1, 102), 'B'): pilot_status_((1,_102),_'B'),
 ((1, 102), 'C'): pilot_status_((1,_102),_'C'),
 ((1, 102), 'D'): pilot_status_((1,_102),_'D'),
 ((1, 102), 'E'): pilot_status_((1,_102),_'E'),
 ((2, 201), 'A'): pilot_status_((2,_201),_'A'),
 ((2, 201), 'B'): pilot_status_((2,_201),_'B'),
 ((2, 201), 'C'): pilot_status_((2,_201),_'C'),
 ((2, 201), 'D'): pilot_status_((2,_201),_'D'),
 ((2, 201), 'E'): pilot_status_((2,_201),_'E'),
 ((3, 301), 'A'): pilot_status_((3,_301),_'A'),
 ((3, 301), 'B'): pilot_status_((3,_301),_'B'),
 ((3, 301), 'C'): pilot_status_((3,_301),_'C'),
 ((3, 301), 'D'): pilot_status_((3,_301),_'D'),
 ((3, 301), 'E'): pilot_status_((3,_301),_'E'),
 ((3, 302), 'A'): pilot_status_((3,_302)

We instantiate our model and use LpMinimize as the aim is to minimise costs.

In [8]:
model = pulp.LpProblem("PilotMinSchedProb", pulp.LpMinimize)

In our objective function we include our 2 costs: 
* Our variable costs is the product of the variable costs per unit and production
* Our fixed costs is the factory status - 1 (on) or 0 (off) - multiplied by the fixed cost of production

In [9]:
model += pulp.lpSum(
  1
)

In [10]:
model

PilotMinSchedProb:
MINIMIZE
1
VARIABLES

We build up our constraints

In [11]:
for line in lines.index:
    model += sum(pilot_status[(line,x)]*pilot_quals.loc[x,'QUAL'] for x in pilots.index) >= lines.loc[line, 'Requirement']

In [12]:
model

PilotMinSchedProb:
MINIMIZE
1
SUBJECT TO
_C1: 2 pilot_status_((1,_101),_'A') + pilot_status_((1,_101),_'B')
 + 2 pilot_status_((1,_101),_'C') + pilot_status_((1,_101),_'D')
 + 2 pilot_status_((1,_101),_'E') >= 2

_C2: 2 pilot_status_((1,_102),_'A') + pilot_status_((1,_102),_'B')
 + 2 pilot_status_((1,_102),_'C') + pilot_status_((1,_102),_'D')
 + 2 pilot_status_((1,_102),_'E') >= 1

_C3: 2 pilot_status_((2,_201),_'A') + pilot_status_((2,_201),_'B')
 + 2 pilot_status_((2,_201),_'C') + pilot_status_((2,_201),_'D')
 + 2 pilot_status_((2,_201),_'E') >= 2

_C4: 2 pilot_status_((3,_301),_'A') + pilot_status_((3,_301),_'B')
 + 2 pilot_status_((3,_301),_'C') + pilot_status_((3,_301),_'D')
 + 2 pilot_status_((3,_301),_'E') >= 2

_C5: 2 pilot_status_((3,_302),_'A') + pilot_status_((3,_302),_'B')
 + 2 pilot_status_((3,_302),_'C') + pilot_status_((3,_302),_'D')
 + 2 pilot_status_((3,_302),_'E') >= 1

_C6: 2 pilot_status_((4,_401),_'A') + pilot_status_((4,_401),_'B')
 + 2 pilot_status_((4,_401),_'C'

We then solve the model

In [13]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

Let's take a look at the optimal production schedule output for each month from each factory. For ease of viewing we'll output the data to a pandas DataFrame.

In [14]:
pilot_status

{((1, 101), 'A'): pilot_status_((1,_101),_'A'),
 ((1, 101), 'B'): pilot_status_((1,_101),_'B'),
 ((1, 101), 'C'): pilot_status_((1,_101),_'C'),
 ((1, 101), 'D'): pilot_status_((1,_101),_'D'),
 ((1, 101), 'E'): pilot_status_((1,_101),_'E'),
 ((1, 102), 'A'): pilot_status_((1,_102),_'A'),
 ((1, 102), 'B'): pilot_status_((1,_102),_'B'),
 ((1, 102), 'C'): pilot_status_((1,_102),_'C'),
 ((1, 102), 'D'): pilot_status_((1,_102),_'D'),
 ((1, 102), 'E'): pilot_status_((1,_102),_'E'),
 ((2, 201), 'A'): pilot_status_((2,_201),_'A'),
 ((2, 201), 'B'): pilot_status_((2,_201),_'B'),
 ((2, 201), 'C'): pilot_status_((2,_201),_'C'),
 ((2, 201), 'D'): pilot_status_((2,_201),_'D'),
 ((2, 201), 'E'): pilot_status_((2,_201),_'E'),
 ((3, 301), 'A'): pilot_status_((3,_301),_'A'),
 ((3, 301), 'B'): pilot_status_((3,_301),_'B'),
 ((3, 301), 'C'): pilot_status_((3,_301),_'C'),
 ((3, 301), 'D'): pilot_status_((3,_301),_'D'),
 ((3, 301), 'E'): pilot_status_((3,_301),_'E'),
 ((3, 302), 'A'): pilot_status_((3,_302)

In [15]:
output = []
for GO, PILOT in pilot_status:
    var_output = {
        'GO': GO,
        'PILOT': PILOT,
        'FLYING': pilot_status[(GO, PILOT)].varValue
    }
    output.append(var_output)

In [17]:
output_df = pd.DataFrame.from_records(output).sort_values(['GO', 'PILOT'])
output_df.set_index(['GO', 'PILOT'], inplace=True)
print(output_df)

                FLYING
GO       PILOT        
(1, 101) A         1.0
         B         1.0
         C         1.0
         D         1.0
         E         1.0
...                ...
(9, 902) A         0.0
         B         0.0
         C         0.0
         D         0.0
         E         1.0

[70 rows x 1 columns]


Notice above that the factory status is 0 when not producing and 1 when it is producing

In [18]:
output

[{'GO': (1, 101), 'PILOT': 'A', 'FLYING': 1.0},
 {'GO': (1, 101), 'PILOT': 'B', 'FLYING': 1.0},
 {'GO': (1, 101), 'PILOT': 'C', 'FLYING': 1.0},
 {'GO': (1, 101), 'PILOT': 'D', 'FLYING': 1.0},
 {'GO': (1, 101), 'PILOT': 'E', 'FLYING': 1.0},
 {'GO': (1, 102), 'PILOT': 'A', 'FLYING': 1.0},
 {'GO': (1, 102), 'PILOT': 'B', 'FLYING': 0.0},
 {'GO': (1, 102), 'PILOT': 'C', 'FLYING': 0.0},
 {'GO': (1, 102), 'PILOT': 'D', 'FLYING': 0.0},
 {'GO': (1, 102), 'PILOT': 'E', 'FLYING': 0.0},
 {'GO': (2, 201), 'PILOT': 'A', 'FLYING': 1.0},
 {'GO': (2, 201), 'PILOT': 'B', 'FLYING': 1.0},
 {'GO': (2, 201), 'PILOT': 'C', 'FLYING': 1.0},
 {'GO': (2, 201), 'PILOT': 'D', 'FLYING': 1.0},
 {'GO': (2, 201), 'PILOT': 'E', 'FLYING': 1.0},
 {'GO': (3, 301), 'PILOT': 'A', 'FLYING': 1.0},
 {'GO': (3, 301), 'PILOT': 'B', 'FLYING': 1.0},
 {'GO': (3, 301), 'PILOT': 'C', 'FLYING': 1.0},
 {'GO': (3, 301), 'PILOT': 'D', 'FLYING': 1.0},
 {'GO': (3, 301), 'PILOT': 'E', 'FLYING': 1.0},
 {'GO': (3, 302), 'PILOT': 'A', 'FLYING'